In [ ]:
from tensorscaling import scale, unit_tensor, random_tensor, marginal
import numpy as np

# Tensor scaling

Scale 3x3x3 unit tensor to certain non-uniform marginals:

In [ ]:
shape = [3, 3, 3]
targets = [(.5, .25, .25), (.4, .3, .3), (.7, .2, .1)]

res = scale(unit_tensor(3, 3), targets, eps=1e-4)
res

Result(success=True, iterations=70, max_dist=9.733676268599664e-05, ..., log_cap=-0.41047599457753503)

We can also access the scaling matrices and the final scaled state:

In [ ]:
print(res.gs[0], "\n")
print(res.gs[1], "\n")
print(res.gs[2])

[[ 1.50242442+0.j          0.        +0.j          0.        +0.j        ]
 [ 0.37043187-0.20500871j  0.7536671 +0.j          0.        +0.j        ]
 [-0.26107473-0.06307719j -0.24417391-0.53589336j  0.89685076+0.j        ]] 

[[0.85030944+0.j         0.        +0.j         0.        +0.j        ]
 [0.19923981+0.50541438j 1.69495393+0.j         0.        +0.j        ]
 [0.08709811-0.86078352j 0.71820134-0.73180974j 0.97426162+0.j        ]] 

[[ 1.90259547+0.j          0.        +0.j          0.        +0.j        ]
 [-0.57315403-0.15937513j  0.57218426+0.j          0.        +0.j        ]
 [-0.05337434+0.05188302j -0.4574125 -0.03243961j  0.30314931+0.j        ]]


Let's now check that the W tensor *cannot* be scaled to uniform marginals:

In [ ]:
shape = [2, 2, 2, 2]
W = np.zeros(shape)
W[1, 0, 0, 0] = W[0, 1, 0, 0] = W[0, 0, 1, 0] = W[0, 0, 0, 1] = .5
targets = [(.5, .5)] * 4

scale(W, targets, eps=1e-4, max_iterations=1000)

Result(success=False, iterations=1000, max_dist=0.05390983862184736, ..., log_cap=None)

To see more clearly what is going on, we can set the `verbose` flag:

In [ ]:
res = scale(W, targets, eps=1e-4, max_iterations=10, verbose=True)

scaling tensor of shape (2, 2, 2, 2) and type float64
target spectra:
  0: (0.5, 0.5)
  1: (0.5, 0.5)
  2: (0.5, 0.5)
  3: (0.5, 0.5)
#000: max_dist = 0.35355339 @ sys = 3
#001: max_dist = 0.47140452 @ sys = 0
#002: max_dist = 0.56568542 @ sys = 1
#003: max_dist = 0.62853936 @ sys = 2
#004: max_dist = 0.58232323 @ sys = 3
#005: max_dist = 0.59305730 @ sys = 0
#006: max_dist = 0.59545834 @ sys = 1
#007: max_dist = 0.59262283 @ sys = 2
#008: max_dist = 0.59353004 @ sys = 3
#009: max_dist = 0.59357133 @ sys = 0
#010: max_dist = 0.59340661 @ sys = 1
did not converge!


We see that at each point in the algorithm, one of the marginals has Frobenius distance $\approx 0.59$ to being uniform. Indeed, we know that the entanglement polytope of the W tensor does not include the point corresponding to uniform marginals -- see [here](https://www.entanglement-polytopes.org/four_qubits) for an interactive visualization!

# Tuples of matrices and tensors

We can just as well only prescribe the desired spectra for subsystems.
Note that prescribing two out of three marginals amounts to *operator scaling*.

In [ ]:
shape = [3, 3, 3]
targets = [(.4, .3, .3), (.7, .2, .1)]

res = scale(unit_tensor(3, 3), targets, eps=1e-6)
res

Result(success=True, iterations=27, max_dist=9.12607318362196e-07, ..., log_cap=-0.1558365486541544)

Indeed, the last two marginals are as prescribed, while the first marginal is arbitrary.

In [ ]:
print(marginal(res.psi, 0).round(5), "\n")
print(marginal(res.psi, 1).round(5), "\n")
print(marginal(res.psi, 2).round(5))

[[0.31136-0.j      0.01   +0.00929j 0.00949+0.00597j]
 [0.01   -0.00929j 0.36243+0.j      0.02025+0.00254j]
 [0.00949-0.00597j 0.02025-0.00254j 0.32621-0.j     ]] 

[[ 0.4+0.j  0. -0.j -0. +0.j]
 [ 0. +0.j  0.3+0.j -0. +0.j]
 [-0. -0.j -0. -0.j  0.3+0.j]] 

[[ 0.7-0.j -0. -0.j  0. +0.j]
 [-0. +0.j  0.2-0.j  0. -0.j]
 [ 0. +0.j  0. -0.j  0.1+0.j]]
